In [26]:
import numpy as np
from DTClassifier import DTClassifier as DTC
from Feature_Engineering import datatransform
import train_test_split as tts
import metrics

In [27]:
file = './train.csv'
X_train, y_train = datatransform(file)

In [28]:
X_train, y_train, X_test, y_test = tts.holdout(X_train, y_train)

In [29]:
weight = np.ones(len(y_train))/len(y_train)

In [30]:
classifiers = []
a_history = []
weight_history = []

In [31]:
%%time
max_iter = 10
for i in range(max_iter):
    dtc = DTC(weight=weight, min_samples=9, max_gini=0.3)
    dtc.fit(X_train,y_train)
    y_pred = dtc.predict(X_train, negative='True')
    weight_history.append(weight)
    e = np.sum(weight*(y_pred!=y_train))
    a = np.log((1-e)/e + 1e-7)/2
    L = np.exp(-a * y_pred * y_train)
    z = np.sum(weight * L)
    a_history.append(a)
    classifiers.append(dtc)
    weight = weight * L / z
    print('weight\n',np.sum(weight))
    print(dtc.score(X_train,y_train),'\n',i,weight[:5])

weight
 1.0000000000000002
0.815112540192926 
 0 [0.00184013 0.00184013 0.00087638 0.00184013 0.00184013]
weight
 0.9999999999999998
0.8102893890675241 
 1 [0.00199257 0.00199257 0.00047561 0.00199257 0.00199257]
weight
 1.0
0.77491961414791 
 2 [0.00209965 0.00209965 0.00026601 0.00209965 0.00209965]
weight
 0.9999999999999999
0.7588424437299035 
 3 [0.00217305 0.00217305 0.00014993 0.00217305 0.00217305]
weight
 0.9999999999999999
0.7331189710610932 
 4 [2.21944379e-03 2.21944379e-03 8.15203496e-05 2.21944379e-03
 2.21944379e-03]
weight
 1.0
0.7636655948553055 
 5 [2.26893224e-03 2.26893224e-03 3.98193347e-05 2.26893224e-03
 2.26893224e-03]
weight
 1.0
0.6961414790996785 
 6 [2.29457289e-03 2.29457289e-03 2.10278834e-05 2.29457289e-03
 2.29457289e-03]
weight
 1.0
0.7234726688102894 
 7 [2.31459451e-03 2.31459451e-03 1.11863842e-05 2.31459451e-03
 2.31459451e-03]
weight
 1.0
0.6881028938906752 
 8 [2.32698917e-03 2.32698917e-03 6.01481000e-06 2.32698917e-03
 2.32698917e-03]
weight
 1.

In [32]:
np.sum(weight_history)

10.0

In [33]:
y_pred = 0
for i in range(len(classifiers)):
    classifier = classifiers[i]
    y_predi = classifier.predict(X_test)
    for j in y_predi:
        if j==0:
            j = -1
    y_predi = y_predi * a_history[i]
    y_pred += y_predi
y_pred = np.array(y_pred>0, dtype='int')
np.sum(y_pred == y_test)/len(y_test)

0.700374531835206